In [1]:
import torch
import numpy as np
from skimage.measure import marching_cubes
from pytorch3d.structures import Meshes
from pytorch3d.io import save_obj
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    MeshRenderer,
    MeshRasterizer,
    SoftPhongShader
)

from pytorch3d.renderer.mesh.textures import Textures
from shape_sampler import ShapeSampler
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

from mosaic_sdf_visualizer import MosaicSDFVisualizer
from utils_vis import copy_spheres_to_points

In [2]:
import point_cloud_utils as pcu
from pytorch3d.io import load_obj

# Load vertices and faces for a mesh
device = 'cuda'
cow_mesh_path = 'data/cow_mesh/cow.obj'
airplane_mesh_path = 'data/11803_Airplane_v1_l1.obj'
mesh_path = airplane_mesh_path


In [3]:
shape_sampler = ShapeSampler.from_file(mesh_path, device='cuda',     
                                        normalize_shape=True,
                                       make_watertight=True)


/home/che/miniforge3/envs/gen3d/lib/python3.10/site-packages/point_cloud_utils/_mesh_io.py:338: RuntimeWarning: TinyObjReader: Both `d` and `Tr` parameters defined for "11803_Airplane_body". Use the value of `d` for dissolve (line 8 in .mtl.)
Both `d` and `Tr` parameters defined for "11803_Airplane_wing_R". Use the value of `d` for dissolve (line 22 in .mtl.)
Both `d` and `Tr` parameters defined for "11803_Airplane_wing_details_R". Use the value of `d` for dissolve (line 36 in .mtl.)
Both `d` and `Tr` parameters defined for "11803_Airplane_tail". Use the value of `d` for dissolve (line 50 in .mtl.)
Both `d` and `Tr` parameters defined for "11803_Airplane_wing_details_L". Use the value of `d` for dissolve (line 64 in .mtl.)
Both `d` and `Tr` parameters defined for "11803_Airplane_wing_L". Use the value of `d` for dissolve (line 78 in .mtl.)

  mesh_dict = load_mesh_internal(mesh_filename, dtype)


In [4]:
p = torch.tensor([
    [-.99,-.99,-.99],
    [0,0,0],
    [.5,.5,.5]
], dtype=torch.float32)

shape_sampler(p)

tensor([ 1.2156, -0.0997,  0.5619])

In [5]:
shape_sampler.vertices.max(axis=0)

torch.return_types.max(
values=tensor([0.9491, 1.0000, 0.3834], device='cuda:0', dtype=torch.float64),
indices=tensor([23071, 14200,  7655], device='cuda:0'))

In [ ]:

v_pcu, f_pcu = pcu.load_mesh_vf(mesh_path)
v_pcu -= np.array([.1,0,0])

source_mesh_pcu = MosaicSDFVisualizer.create_mesh_from_verts(v_pcu, f_pcu, vert_colors = [.8, 0, 0], device=device)

vertices_pt, faces_pt, aux = load_obj(mesh_path, device=device)
vertices_pt += np.array([.1,0,0])
source_mesh_pt = MosaicSDFVisualizer.create_mesh_from_verts(vertices_pt, faces_pt.verts_idx, vert_colors = [0, 0, 0], device=device)

# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "mesh_pcu": source_mesh_pcu,
        "mesh_pt": source_mesh_pt
    }
})
fig.show()    

In [ ]:
resolution = 20_000
rand_samples = 1024
v_watertight, f_watertight = pcu.make_mesh_watertight(v_pcu, f_pcu, resolution=resolution)

rand_points_on_surface_idx = torch.randperm(resolution)[:rand_samples]
rand_points_on_surface = v_watertight[rand_points_on_surface_idx]

wt_mesh = MosaicSDFVisualizer.create_mesh_from_verts(v_watertight, f_watertight, vert_colors = [0, 0, .8], device=device)

# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "mesh": wt_mesh
    }
})
fig.show() 

In [ ]:
copy_spheres_to_points(rand_points_on_surface, source_mesh_pcu, device)


In [ ]:
# rand_samples = v_watertight.shape[0] * 40

f_i, bc = pcu.sample_mesh_random(v_watertight, f_watertight, num_samples=rand_samples,
                                 random_seed=42)

# Use the face indices and barycentric coordinate to compute sample positions and normals
v_sampled = pcu.interpolate_barycentric_coords(f_watertight, f_i, bc, v_watertight)
v_sampled

In [ ]:
def create_mesh_from_verts(vertices, faces, device, vert_colors=[1,0,0]):
    total_verts = vertices.shape[0]
    verts_rgb = torch.ones((1, total_verts, 3), device=device)  # White color for all vertices
    verts_rgb *= torch.tensor(vert_colors, device=device)
    # Initialize the textures with the corrected verts_rgb
    textures = Textures(verts_rgb=verts_rgb)

    # Create the mesh
    return Meshes(verts=[vertices.to(device)], faces=[faces.to(device)], 
                    textures=textures).to(device)


In [ ]:
from pysdf import SDF

# Load some mesh (don't necessarily need trimesh)
import trimesh
o = trimesh.load('data/cow_mesh/cow.obj')


device = 'cpu'

verts = torch.tensor(o.vertices.copy(), dtype=torch.float32)
verts, target_mean, target_max_vertex = ShapeSampler.normalize_vertices(verts)

f = SDF(verts.numpy(), o.faces); # (num_vertices, 3) and (num_faces, 3)

faces = torch.tensor(o.faces.copy(), dtype=torch.int64)
cow_mesh = create_mesh_from_verts(verts, faces, device, [0, 0, 1])

print(verts.min(axis=0))
print(verts.max(axis=0))

# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "mesh": cow_mesh
    }
})
fig.show()    
# # Compute some SDF values (negative outside);
# # takes a (num_points, 3) array, converts automatically
# origin_sdf = f([0, 0, 0])
# sdf_multi_point = f([[0, 0, 0],[1,1,1],[0.1,0.2,0.2]])

# # Contains check
# origin_contained = f.contains([0, 0, 0])

# # Misc: nearest neighbor
# origin_nn = f.nn([0, 0, 0])

# # Misc: uniform surface point sampling
# random_surface_points = f.sample_surface(10000)

# # Misc: surface area
# the_surface_area = f.surface_area

In [ ]:
sdf_mesh = MosaicSDFVisualizer.rasterize_sdf(sdf_func=f, resolution=16,
                                             sdf_scaler=-1, extra_sdf_offset=[1,0,0])


In [ ]:
# Initialize a renderer
R, T = look_at_view_transform(5, 45, 15)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
)

# Render the mesh


with torch.no_grad():
    images = renderer(sdf_mesh.to(device))

    # vis.shape
    plt.figure(figsize=(4, 4))
    plt.imshow(images[0, ..., :3].cpu().numpy())
    plt.axis("off")

# images


In [ ]:
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "mesh": sdf_mesh,
        "cow": cow_mesh
    }
})
fig.show()    